In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
# Split the dataset into features (X) and target (y)
X = data.drop(columns=['DEATH_EVENT'])
y = data['DEATH_EVENT']

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Fit the Logistic Regression model
#log_reg = LogisticRegression(random_state=42)
model = RandomForestClassifier(random_state=43)
model.fit(X_train,y_train)
temp = pd.DataFrame(model.predict_proba(X_train).tolist(), columns=model.classes_)
temp['max_prob'] = temp.max(axis=1)

#log_reg.fit(X_train_scaled, y_train)

# Predict on the test set
#y_pred = log_reg.predict(X_test_scaled)
y_rf = model.predict(X_test)
# Calculate the accuracy score
#accuracy = accuracy_score(y_test, y_pred)

idx = temp[temp.max_prob >= .9].index
accuracy2 = accuracy_score(y_test.iloc[idx], y_rf[idx])


accuracy2


IndexError: positional indexers are out-of-bounds

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')

# Split the dataset into features (X) and target (y)
X = data.drop(columns=['DEATH_EVENT'])
y = data['DEATH_EVENT']

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the Random Forest Classifier model
model = RandomForestClassifier(random_state=43)
model.fit(X_train_scaled, y_train)

# Predict probabilities on the test set
probs = model.predict_proba(X_test_scaled)

# Convert the predicted probabilities to a DataFrame
probs_df = pd.DataFrame(probs, columns=model.classes_)

# Add a column for the maximum predicted probability
probs_df['max_prob'] = probs_df.max(axis=1)

# Get the predicted class using the maximum probability
probs_df['pred_class'] = model.predict(X_test_scaled)

# Set a probability threshold (e.g., 0.9 for high-confidence predictions)
threshold = 0.9275

# Filter rows where the maximum probability is above the threshold
high_confidence_idx = probs_df[probs_df['max_prob'] >= threshold].index
data2 = pd.DataFrame(data)
newData = data2.loc[high_confidence_idx]
newData = data2.drop(data2.index.difference(high_confidence_idx))

# Get the high-confidence predictions and their corresponding true labels
high_confidence_preds = probs_df.loc[high_confidence_idx, 'pred_class']
high_confidence_true = y_test.iloc[high_confidence_idx]

# Calculate the accuracy for high-confidence predictions
accuracy_high_confidence = accuracy_score(high_confidence_true, high_confidence_preds)


accuracy_high_confidence , len(high_confidence_idx)/len(y_test)



(1.0, 0.2833333333333333)

#Heart Failure Prediction Model

##Loading Data

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
# Load the data
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
data

data.dropna()

data.isnull().sum() , data.describe()




(age                         0
 anaemia                     0
 creatinine_phosphokinase    0
 diabetes                    0
 ejection_fraction           0
 high_blood_pressure         0
 platelets                   0
 serum_creatinine            0
 serum_sodium                0
 sex                         0
 smoking                     0
 time                        0
 DEATH_EVENT                 0
 dtype: int64,
               age     anaemia  creatinine_phosphokinase    diabetes  \
 count  299.000000  299.000000                299.000000  299.000000   
 mean    60.833893    0.431438                581.839465    0.418060   
 std     11.894809    0.496107                970.287881    0.494067   
 min     40.000000    0.000000                 23.000000    0.000000   
 25%     51.000000    0.000000                116.500000    0.000000   
 50%     60.000000    0.000000                250.000000    0.000000   
 75%     70.000000    1.000000                582.000000    1.000000   
 max  

##Splitting Data

In [95]:
# Separate features and target 
X = data.drop(columns=['DEATH_EVENT'])
y = data['DEATH_EVENT']

# Split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.61, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Predict on the test set using the best model
model = LogisticRegression()
model.fit(X_train_scaled,y_train)
y_pred = model.predict(X_test_scaled)


# time (0.375) positive 
# serum_creatinine (0.155)
# ejection_fraction (0.112)
# creatinine_phosphokinase (0.083)
# age (0.078)
# platelets (0.075)


# Evaluate model performance with multiple metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")


Accuracy: 0.8525
Precision: 0.8491
Recall: 0.7031
F1-score: 0.7692
ROC AUC: 0.8179


##Saving The Model

In [96]:
import joblib

# Save the trained model
joblib.dump(model, 'heart_failure_model.pkl')

['heart_failure_model.pkl']

##Loading The model

In [ ]:
# Load the saved model
loaded_model = joblib.load('heart_failure_model.pkl')

# Prepare new data for prediction (ensure it has the same features as the training data)
new_data = ...  # Your new data

# Make predictions
predictions = loaded_model.predict(new_data)